In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from wrangle_xml import wrangle_xml, preprocessing_xml

In [3]:
df = wrangle_xml()

In [4]:
df.head(2)

,creationDate,device,endDate,sourceName,sourceVersion,startDate,type,unit,value
0,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52 -0500,Noah's iPhone,9.3.1,2016-06-03 16:40:09 -0500,HKQuantityTypeIdentifierStepCount,count,70.0
1,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26 -0500,Noah's iPhone,9.3.1,2016-06-03 16:57:23 -0500,HKQuantityTypeIdentifierStepCount,count,68.0


In [5]:
df = preprocessing_xml(df)

In [6]:
df.head(2)

,creationDate,device,endDate,sourceName,sourceVersion,type,unit,value
startDate,,,,,,,,
2016-06-03 16:40:09,2016-06-03 17:38:46,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52,Noah's iPhone,9.3.1,HKQuantityTypeIdentifierStepCount,count,70.0
2016-06-03 16:57:23,2016-06-03 17:38:46,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26,Noah's iPhone,9.3.1,HKQuantityTypeIdentifierStepCount,count,68.0


In [7]:
df_2 = pd.read_csv("Export.csv")

In [8]:
df_2['Date'] = pd.to_datetime(df_2.Date)

In [9]:
df_2.Date.agg([min, max])

min   2018-08-26
max   2020-08-16
Name: Date, dtype: datetime64[ns]

In [10]:
df_2.Date > "2020-01-01"

0       True
1       True
2       True
3       True
4       True
       ...  
584    False
585    False
586    False
587    False
588    False
Name: Date, Length: 589, dtype: bool

In [11]:
df.type.value_counts()

HKQuantityTypeIdentifierStepCount                 67915
HKQuantityTypeIdentifierDistanceWalkingRunning    67908
HKQuantityTypeIdentifierFlightsClimbed             1201
Name: type, dtype: int64

In [12]:
df = pd.read_csv("xml.csv")

In [13]:
df.type.value_counts()

HKQuantityTypeIdentifierStepCount                 67915
HKQuantityTypeIdentifierDistanceWalkingRunning    67908
HKQuantityTypeIdentifierFlightsClimbed             1201
Name: type, dtype: int64

In [24]:
df = wrangle_xml()

In [117]:
df_steps = df[df.type == 'HKQuantityTypeIdentifierStepCount']

In [118]:
df_steps = preprocessing_xml(df_steps)

/Users/noahsdisk/codeup-data-science/time_series_steps_project/wrangle_xml.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['endDate'] = df.endDate.str.replace("-0500", "")
/Users/noahsdisk/codeup-data-science/time_series_steps_project/wrangle_xml.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['startDate'] = df.startDate.str.replace("-0500", "")
/Users/noahsdisk/codeup-data-science/time_series_steps_project/wrangle_xml.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [119]:
df_steps = df_steps.reset_index()

In [120]:
create = df_steps.set_index('creationDate').resample("D").value.sum()
create = pd.DataFrame(create)
create.reset_index(inplace=True)
create = create.rename(columns={'value': 'create_value'})
create.head()

,creationDate,create_value
0,2016-06-03,469.0
1,2016-06-04,4355.0
2,2016-06-05,4395.0
3,2016-06-06,6535.0
4,2016-06-07,6588.0


In [121]:
start = df_steps.set_index('startDate').resample("D").value.sum()
start = pd.DataFrame(start)
start.reset_index(inplace=True)
start = start.rename(columns={'value': 'start_value'})

In [122]:
end = df_steps.set_index('endDate').resample("D").value.sum()
end= pd.DataFrame(end)
end.reset_index(inplace=True)
end = end.rename(columns={'value': 'end_value'})

In [123]:
df_validate = pd.read_csv("Export.csv")

In [124]:
df_validate['Date'] = pd.to_datetime(df_validate.Date)

In [ ]:
df_validate = (df_validate.merge(start, left_on='Date', right_on='startDate')
    .merge(create, left_on='Date', right_on='creationDate')
    .merge(end, left_on='Date', right_on='endDate')
)

In [128]:
df_validate.drop(columns=(['startDate', 'endDate', 'creationDate']), inplace=True)

In [129]:
df_validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 589 entries, 0 to 588
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             589 non-null    datetime64[ns]
 1   Steps            589 non-null    int64         
 2   Distance         589 non-null    float64       
 3   Floors Ascended  589 non-null    int64         
 4   start_value      589 non-null    float64       
 5   create_value     589 non-null    float64       
 6   end_value        589 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 36.8 KB


In [130]:
df_validate

,Date,Steps,Distance,Floors Ascended,start_value,create_value,end_value
0,2020-08-16,14656,6.7,0,14705.0,14705.0,14705.0
1,2020-08-15,10103,4.7,8,10103.0,10103.0,10103.0
2,2020-08-14,10554,4.9,0,10513.0,10513.0,10513.0
3,2020-08-13,8839,4.2,0,8839.0,8839.0,8839.0
4,2020-08-12,10937,5.2,0,10869.0,10869.0,10869.0
...,...,...,...,...,...,...,...
584,2018-08-30,3816,1.5,0,3797.0,3812.0,3797.0
585,2018-08-29,5965,2.4,0,5984.0,5964.0,5984.0
586,2018-08-28,3590,1.5,1,3571.0,3571.0,3571.0
587,2018-08-27,3147,1.3,1,3166.0,3166.0,3166.0


In [131]:
df_validate.sum()

Steps              5864517.0
Distance              2585.1
Floors Ascended        765.0
start_value        5936360.0
create_value       5936697.0
end_value          5936343.0
dtype: float64

In [133]:
abs(df_validate.Steps - df_validate.start_value).sum() / df_validate.start_value.sum()

0.12700425850184288